<a href="https://colab.research.google.com/github/ZitongZhou/react_inverse/blob/master/make_inputs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pickle as pk
import os
import time
from google.colab import drive
from google.colab import files
import h5py
import matplotlib.pyplot as plt
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
with open('/content/drive/MyDrive/react_inverse/training_data/kds.pkl', 'rb') as file:
    kds = pk.load(file)

In [ ]:
assert len(kds[0])==6 and len(kds[0][0])==41 and len(kds[0][0][0])==81, \
    'size error, should be [6, 41, 81]'+str([len(kds[0]), len(kds[0][0]), len(kds[0][0][0])])

In [ ]:
def simple_plot(c_map, title=''):
    nx = 81
    ny = 41
    Lx = 2500
    Ly = 1250

    x = np.linspace(0, Lx, nx)
    y = np.linspace(0, Ly, ny)
    X,Y = np.meshgrid(x, y)
    fig, axs = plt.subplots(1,1)
#        axs.set_xlabel('x(m)')
#        axs.set_ylabel('y(m)')
    axs.set_xlim(0,Lx)
    axs.set_ylim(0,Ly)
    c01map = axs.imshow(c_map, cmap='jet',
              extent=[x.min(), x.max(), y.min(), y.max()],
              vmin=c_map.min(), vmax = c_map.max(),
              origin='lower')
    fig.colorbar(c01map, ax=axs,shrink=0.62)
    name = title + '.pdf'
    plt.title(title)
#         fig.savefig('images/'+name, format='pdf',bbox_inches='tight')
    plt.show()
    return

## Read output files

In [ ]:
for i in range(5):
  output_file = '/content/drive/MyDrive/react_inverse/simu_outputs/output_'+str(i)+'.hdf5'
  f = h5py.File(output_file, "r")
  conc = np.array(f['concentration'])
  head = np.array(f['head'])
  simple_plot(conc[-1][0], title='conc0')
  simple_plot(conc[-1][1], title = 'conc1')
  simple_plot(conc[-1][2], title = 'conc2')
  simple_plot(conc[-1][3], title = 'conc3')
  simple_plot(conc[-1][4], title = 'conc4')
  simple_plot(conc[-1][5], title = 'conc5')
  f.close()

In [ ]:
len(head)

41

## Make input files

In [ ]:
N = 15000
# local_state = np.random.RandomState(seed = None)

### x
np.random.seed(888)
y_wel = np.array([6, 13, 20, 27, 34,
        6, 13, 20, 27, 34,
        6, 13, 20, 27, 34,
        6, 13, 20, 27, 34])# #np.random.randint(low = 5, high = 35, size = 20)
x_wel = np.array([1, 1, 1, 1, 1, 
         7, 7, 7, 7, 7,
         13, 13, 13, 13, 13,
         20, 20, 20, 20, 20])# #np.random.randint(low = 0, high = 25, size = 20)
wells = {i: [y_wel[i], x_wel[i]] for i in range(len(y_wel))}

x_ind = np.random.choice(20, N)

x = [wells[x_ind[i]] for i in range(N)]

### q
q = 100*np.random.randint(low=1, high=15, size=(N, 5))

### make well stress period data:
con_rate = []
for sim in range(N):
    welspd = []
    for i in range(5):
        welspd.append([3, x[sim][0], x[sim][1], q[sim, i], 2])
    welspd.append([3, x[sim][0], x[sim][1], 0, 2])
    con_rate.append(welspd)
con_rate = np.array(con_rate)

In [ ]:
con_rate[1996]

array([[   3,   27,   20, 1400,   -1],
       [   3,   27,   20,  400,   -1],
       [   3,   27,   20,  100,   -1],
       [   3,   27,   20,  200,   -1],
       [   3,   27,   20,  400,   -1],
       [   3,   27,   20,    0,   -1]])

In [ ]:
N = 5000
for ndata in range(N):
    hf =  h5py.File('/content/drive/MyDrive/react_inverse/simu_inputs/input_{}.hdf5'.format(ndata), 'w')
    hf.create_dataset('kd', data = kds[ndata], dtype ='f', compression = 'gzip')
    hf.create_dataset('welspd', data = con_rate[ndata], dtype ='f', compression = 'gzip')
    hf.close()